# Classify Species using De-noised Melspectrograms and a CNN

Here I will be using my dataset (https://www.kaggle.com/tjcdev/rfcsaddenoisedmelspectrainv1) that was created by converting all the .flac files into de-noised melspectrogram images. 

The following notebook contains the code for converting the .flac files into the de-noised melspectrograms https://www.kaggle.com/tjcdev/convert-audio-to-de-noised-melspectrogram-image

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

import pandas as pd
import numpy as np

import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.models import Model

from tqdm import tqdm

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Data Preprocessing

In [ ]:
BASE_DIR = '../input/rfcsaddenoisedmelspectrainv1/train_images'
OUTPUT_DIR = '.'

### Create two arrays, one of images and one of the species in this image.

In [ ]:
train_fp = pd.read_csv('../input/rfcx-species-audio-detection/train_fp.csv')
train_fp.head()

In [ ]:
NUM_SPECIES = len(set(train_fp['species_id'].values))
print('There are %s species' % (NUM_SPECIES))

In [ ]:
images = []
species = []
for index, row in tqdm(train_fp.iterrows()):    
    # Load the image
    IMG_HEIGHT = 310
    IMG_WIDTH = 154
    image_path= '%s/%s.jpg' % (BASE_DIR, str(row['recording_id']))
    image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255 
    
    # Put the image into an array
    images = images + [image]
    
    # Put the species_id into an array
    species = species + [int(row['species_id'])]

species_dummies = pd.get_dummies(np.array(species))

In [ ]:
species_dummies

The image shapes are (154, 310, 3).

In [ ]:
'''
# Save Images Array
images_file = open('%s/images' % (OUTPUT_DIR), "wb")
np.save(images_file, images)
# close the file
images_file.close

# Save Species Array
species_file = open('%s/species' % (OUTPUT_DIR), "wb")
np.save(species_file, species)
# close the file
species_file.close
'''

# Compile and Fit Model

In [ ]:
# Build the model using the functional API
i = Input(shape=(IMG_HEIGHT,IMG_WIDTH, 3))
x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(NUM_SPECIES, activation='softmax')(x)

model = Model(i, x)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x=np.array(images, np.float32), y=np.array(species_dummies, np.float32), epochs=15)

# Predict

In [ ]:
model.predict(np.array(images[:2], np.float32))

# Future Work

Next I will build a prediction pipeline on the test set. In the meantime I hope this notebook is a helpful example of how to use the denoised melspectrogram dataset.